## Load comments, labels and stop_words

In [1]:
import pickle
import numpy as np

In [4]:
with open('./labels.pkl', 'rb') as f:
    labels = pickle.load(f)

with open('./comments.pkl', 'rb') as f:
    comments = pickle.load(f)

stop_list = []    
# with open('./stop_words.txt', 'r') as f:
#     lines = f.readlines()
#     for line in lines:
#         line = line.replace('\n', '')
#         stop_list.append(line)

In [5]:
len(stop_list)

0

In [6]:
len(labels), len(comments)

(50000, 50000)

## Onehot labels, nparray

In [7]:
def one_hot(y, n_classes):
    y_res = []
    for i in range(len(y)):
        input_matrix = np.asarray(y[i])
        y_onehot = np.eye(n_classes)[input_matrix]
        y_res.append(y_onehot)
    return y_res

In [8]:
labels = one_hot(labels, 2)

In [9]:
labels, comments = np.array(labels), np.array(comments)

## Shuffle and split train/test

In [10]:
from sklearn.utils import shuffle

labels, comments = shuffle(labels, comments, random_state=1)

/Users/phamhoanganh/virtualenv/python3_custom/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/phamhoanganh/virtualenv/python3_custom/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/phamhoanganh/virtualenv/python3_custom/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [11]:
X_train = comments[0:49000]
y_train = labels[0:49000]

X_test = comments[49000:50000]
y_test = labels[49000:50000]

# TFIDF

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

/Users/phamhoanganh/virtualenv/python3_custom/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [43]:
vectorizer = TfidfVectorizer(max_features=512, stop_words=stop_list)

In [44]:
x_train = vectorizer.fit_transform(X_train)

In [45]:
x_test = vectorizer.transform(X_test)

In [ ]:
words = vectorizer.get_feature_names()
words

In [34]:
# with open("stop_words.txt", "a+") as f:
#     for word in words:
#         f.write(word + '\n')
# f.close()

In [47]:
x_train[0].shape

(1, 512)

In [78]:
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

## Training Deeplearning

In [37]:
from keras.models import Sequential
from keras import backend as K
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, ELU, ZeroPadding2D, Reshape, Permute
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import keras
from keras.layers.recurrent import GRU

In [48]:
input_shape = (512, )
opt = Adam(lr = 1e-4)

batch_size = 128
nb_epochs = 20

In [40]:
print('Build CNN model ...')

model = Sequential()
model.add(Dense(128, input_shape=input_shape))
model.add(Dropout(0.2))
model.add(Dense(units=64))
model.add(Dropout(0.2))
model.add(Dense(units=16))
model.add(Dropout(0.5))
model.add(Dense(units=2, activation='softmax'))

Build CNN model ...


In [41]:
print("Compiling ...")
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Compiling ...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 16)                1040      
_________________________________________________________________
dropout_6 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 34     

In [42]:
print("Training ...")
model.fit(x_train, y_train, batch_size=batch_size, epochs=nb_epochs, validation_data=(x_test, y_test))

Training ...
Train on 49000 samples, validate on 1000 samples
Epoch 1/20
49000/49000 [==============================] - 2s 33us/step - loss: 0.5737 - acc: 0.7156 - val_loss: 0.4155 - val_acc: 0.8280
Epoch 2/20
49000/49000 [==============================] - 1s 22us/step - loss: 0.4223 - acc: 0.8230 - val_loss: 0.3538 - val_acc: 0.8450
Epoch 3/20
49000/49000 [==============================] - 1s 22us/step - loss: 0.3984 - acc: 0.8334 - val_loss: 0.3459 - val_acc: 0.8490
Epoch 4/20
49000/49000 [==============================] - 1s 21us/step - loss: 0.3921 - acc: 0.8346 - val_loss: 0.3460 - val_acc: 0.8450
Epoch 5/20
49000/49000 [==============================] - 1s 22us/step - loss: 0.3883 - acc: 0.8387 - val_loss: 0.3453 - val_acc: 0.8500
Epoch 6/20
49000/49000 [==============================] - 1s 23us/step - loss: 0.3876 - acc: 0.8372 - val_loss: 0.3438 - val_acc: 0.8540
Epoch 7/20
49000/49000 [==============================] - 1s 23us/step - loss: 0.3861 - acc: 0.8385 - val_loss: 0.34

In [107]:
# model.save("emotion.h5")

# Training with DecisionTree

In [49]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import metrics

efc = ExtraTreesClassifier(n_estimators=512)
print("Training...")
efc.fit(x_train, y_train)
print("Training done! Testing....")
y_pred = efc.predict(x_test)
score_test = metrics.accuracy_score(y_test, y_pred)
print(score_test)

/Users/phamhoanganh/virtualenv/python3_custom/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/phamhoanganh/virtualenv/python3_custom/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Training...
Training done! Testing....
0.869


In [96]:
from pyvi import ViTokenizer
text = "sản phẩm kém"
token = ViTokenizer.tokenize(text.lower())

In [97]:
efc.predict(vectorizer.transform([token]))

array([[1., 0.]])

In [95]:
with open('efc.pkl', 'wb') as f:
    pickle.dump(efc, f)